In [53]:
#%matplotlib inline

In [99]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import time as t
from sklearn import preprocessing as pre

path="/home/stan/Documents/dev/telstra/data/"

trainFile="train.csv"
testFile="test.csv"
resourceFile="resource_type.csv"
eventTypeFile="event_type.csv"
logFeatureFile="log_feature.csv"
severityTypeFile="severity_type.csv"

test=pd.read_csv(filepath_or_buffer=path+testFile,delimiter=",",header=0)
train=pd.read_csv(filepath_or_buffer=path+trainFile,delimiter=",",header=0)

resource=pd.read_csv(filepath_or_buffer=path+resourceFile,delimiter=",",header=0)
event=pd.read_csv(filepath_or_buffer=path+eventTypeFile,delimiter=",",header=0)
feature=pd.read_csv(filepath_or_buffer=path+logFeatureFile,delimiter=",",header=0)
severity=pd.read_csv(filepath_or_buffer=path+severityTypeFile,delimiter=",",header=0)
join=pd.DataFrame({'id':[]})
datasets={'train':train,'test':test,'resource':resource,'event':event,'feature':feature,'severity':severity}
notCat=['id','volume']

cat_cols=['location','log_feature','severity_type','resource_type','event_type']
#creates join dataset table that will store features for train and test datasets
datasets['train']['sample']='train'
datasets['test']['sample']='test'
datasets['test']['fault_severity']=np.nan
join=pd.concat([datasets['train'],datasets['test']],ignore_index=True)
samples=['train','test']
for key, dataset in datasets.items():
    if key not in samples:
        join=pd.merge(join,dataset,on='id',how='inner')

In [100]:
join.head()

,fault_severity,id,location,sample,severity_type,event_type,resource_type,log_feature,volume
0,1,14121,location 118,train,severity_type 2,event_type 34,resource_type 2,feature 312,19
1,1,14121,location 118,train,severity_type 2,event_type 34,resource_type 2,feature 232,19
2,1,14121,location 118,train,severity_type 2,event_type 35,resource_type 2,feature 312,19
3,1,14121,location 118,train,severity_type 2,event_type 35,resource_type 2,feature 232,19
4,0,9320,location 91,train,severity_type 2,event_type 34,resource_type 2,feature 315,200


In [111]:
from scipy.stats import chisquare
from statsmodels.graphics.mosaicplot import mosaic
df=pd.crosstab(join['log_feature'],join['event_type'],margins=True)
df
#f,p=chisquare(df,axis=1)
#f

event_type,event_type 1,event_type 10,event_type 11,event_type 12,event_type 13,event_type 14,event_type 15,event_type 17,event_type 18,event_type 19,...,event_type 50,event_type 51,event_type 52,event_type 53,event_type 54,event_type 6,event_type 7,event_type 8,event_type 9,All
log_feature,,,,,,,,,,,,,,,,,,,,,
feature 1,0,0,26,0,0,0,0,0,0,0,...,0,0,0,0,6,0,0,0,0,63
feature 10,0,0,33,0,4,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,153
feature 100,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
feature 101,0,0,18,0,0,0,19,0,0,0,...,0,0,0,0,0,0,0,0,0,37
feature 102,0,0,2,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,4
feature 103,0,0,65,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,68
feature 104,0,3,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26
feature 105,0,0,27,0,0,0,27,0,0,0,...,0,0,0,0,2,0,0,0,0,56
feature 106,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,3


In [96]:
from scipy.stats import chisquare
dict_stats={}
for column in cat_cols:
    df=pd.crosstab(join[column],join['fault_severity'])
    df[column+'_sum']=(df.sum(axis=1))
    df.columns=['0','1','2',column+'_sum']
    df[column+'_std']=df[['0','1','2']].std(axis=1)
    f,_=chisquare(df[['0','1','2']],axis=1)
    df[column+'_pchisqr']=f
    
    fs=[column+'_sum',column+'_std',column+'_pchisqr']
    df=df[fs]
    df[column]=df.index
    dict_stats[column]=df
    join=pd.merge(join,df,on=column,how="inner")

In [97]:
join.iloc[:,8:]=pre.scale(join.iloc[:,8:])

In [98]:
plt.plot(join.log_feature_pchisqr)
plt.show()

In [94]:
join.head()

,fault_severity,id,location,sample,severity_type,event_type,resource_type,log_feature,volume,location_sum,...,log_feature_pchisqr,severity_type_sum,severity_type_std,severity_type_pchisqr,resource_type_sum,resource_type_std,resource_type_pchisqr,event_type_sum,event_type_std,event_type_pchisqr
0,1,14121,location 118,train,severity_type 2,event_type 34,resource_type 2,feature 312,3.4,0.008197,...,-9.906463e-36,-1,1,0,0,0,0,0,0.683451,0
1,0,12008,location 118,train,severity_type 2,event_type 34,resource_type 2,feature 312,2.2,0.008197,...,-9.906463e-36,-1,1,0,0,0,0,0,0.683451,0
2,0,18441,location 118,train,severity_type 2,event_type 34,resource_type 2,feature 312,2.8,0.008197,...,-9.906463e-36,-1,1,0,0,0,0,0,0.683451,0
3,0,9479,location 118,train,severity_type 2,event_type 34,resource_type 2,feature 312,3.0,0.008197,...,-9.906463e-36,-1,1,0,0,0,0,0,0.683451,0
4,0,2627,location 118,train,severity_type 2,event_type 34,resource_type 2,feature 312,3.2,0.008197,...,-9.906463e-36,-1,1,0,0,0,0,0,0.683451,0


In [30]:
join.set_index(join['id'])
join=join.ix[:, 'location_sum':]
join['id']=join.index
join.head()

,location_sum,location_std,location_pchisqr,log_feature_sum,log_feature_std,log_feature_pchisqr,severity_type_sum,severity_type_std,severity_type_pchisqr,resource_type_sum,resource_type_std,resource_type_pchisqr,event_type_sum,event_type_std,event_type_pchisqr,id
0,0.144663,0.186937,7.158741e-76,1,1,0,0.663063,1,0,1,1,0,0.867749,0.946415,0,0
1,0.144663,0.186937,7.158741e-76,1,1,0,0.663063,1,0,1,1,0,0.867749,0.946415,0,1
2,0.144663,0.186937,7.158741e-76,1,1,0,0.663063,1,0,1,1,0,0.867749,0.946415,0,2
3,0.144663,0.186937,7.158741e-76,1,1,0,0.663063,1,0,1,1,0,0.867749,0.946415,0,3
4,0.144663,0.186937,7.158741e-76,1,1,0,0.663063,1,0,1,1,0,0.867749,0.946415,0,4


In [31]:
stats=join

In [32]:
join=pd.concat([datasets['train'],datasets['test']],ignore_index=True)
join.head()

,fault_severity,id,location,sample
0,1,14121,location 118,train
1,0,9320,location 91,train
2,1,14394,location 152,train
3,1,8218,location 931,train
4,0,14804,location 120,train


In [33]:
#convert table of ids and classes to binary row feature vectors for each unique id
for key, dataset in datasets.items():
    tmp=pd.get_dummies(dataset,dummy_na=True)
    tmp=tmp.groupby('id').sum()
    tmp['id']=tmp.index
    dataset=tmp
    datasets[key]=dataset

In [34]:
#joins features from different datasets into one dataframe, features only for evaluated ids are extracted
#(i.e. some features may be not related to any id)
samples=['train','test']
for key, dataset in datasets.items():
    if key not in samples:
        join=pd.merge(join,dataset,on='id',how='left')

join=pd.merge(join,stats,on='id')
join=join.set_index(['sample'])

In [35]:
join=pd.get_dummies(join) #fixes dummies for location

In [36]:
join.head()

,fault_severity,id,severity_type_severity_type 1,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5,severity_type_nan,event_type_event_type 1,event_type_event_type 10,...,location_location 990,location_location 991,location_location 992,location_location 993,location_location 994,location_location 995,location_location 996,location_location 997,location_location 998,location_location 999
sample,,,,,,,,,,,,,,,,,,,,,
train,1,14121,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
train,0,9320,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
train,1,14394,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
train,1,8218,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
train,0,14804,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
#normalizes a continuous feature
#join['volume']=pre.minmax_scale(join['volume'].astype(float))

#fills in missing data
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
join = DataFrameImputer().fit_transform(join)
#join=join.fillna(value=0)

In [50]:
from kmodes import kmodes

km = kmodes.KModes(n_clusters=5, init='Huang', n_init=1, verbose=1)
clusters = km.fit_predict(join[features])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 4520, cost: 167141.0
Run 1, iteration: 2/100, moves: 3732, cost: 163281.0
Run 1, iteration: 3/100, moves: 905, cost: 163281.0


In [72]:
join['clusters']=clusters
join=pd.concat([join,pd.get_dummies(join['clusters'],prefix='cluster')],axis=1)
del join['clusters']
join.head()

,fault_severity,id,p0,p1,p2,sum,volume,log_feature_feature 1,log_feature_feature 10,log_feature_feature 100,...,location_location 995,location_location 996,location_location 997,location_location 998,location_location 999,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4
sample,,,,,,,,,,,,,,,,,,,,,
train,1,14121,0.909091,0.090909,0.000000,33,0.094788,0,0,0,...,0,0,0,0,0,0,0,1,0,0
train,0,9320,0.897959,0.102041,0.000000,49,3.670196,0,0,0,...,0,0,0,0,0,0,0,1,0,0
train,1,14394,0.600000,0.400000,0.000000,5,-0.368214,0,0,0,...,0,0,0,0,0,0,0,1,0,0
train,1,8218,0.696970,0.181818,0.121212,33,-0.110991,0,0,0,...,0,0,0,0,0,0,0,0,0,1
train,0,14804,0.500000,0.500000,0.000000,12,-0.239603,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [38]:
#creates features and targets for final training
features=join.columns.values.tolist()
not_features=['fault_severity', 'id'] #not_features=['fault_severity', 'id'] 
for x in features:
    if "_nan" in x:
        features.remove(x)
        not_features.append(x)
        
for x in not_features:
    if x in features:
        features.remove(x)

In [39]:
train_features = join.loc['train'][features]
train_target = join.loc['train']['fault_severity']

test_features = join.loc['test'][features]

In [45]:
import xgboost as xgb

sz=(train_features.values).shape

train_X = train_features.values[:int(sz[0] * 0.7), :]
train_Y = train_target.values[:int(sz[0]*0.7)]
#wtr = join.loc['train']['sum'].values[:int(sz[0]*0.7)]


test_X = train_features.values[int(sz[0] * 0.7):, :]
test_Y = train_target.values[int(sz[0] * 0.7):]
#wte = join.loc['train']['sum'].values[int(sz[0] * 0.7):]

xg_train = xgb.DMatrix(train_X, label=train_Y,feature_names=train_features.columns) #
#xg_train.set_weight(wtr)
full_xg_train = xgb.DMatrix(train_features, label=train_target.values,feature_names=train_features.columns) #

xg_test = xgb.DMatrix(test_X, label=test_Y,feature_names=test_features.columns) #
#xg_test.set_weight(wte)
full_xg_test = xgb.DMatrix(test_features.values,feature_names=test_features.columns) #

In [71]:
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 1
param['gamma']= 4
param['max_depth'] = 6
param['min_child_weight']=4
param['max_delta_step']=4

param['nthread'] = 3
#param['subsample']=1
param['num_class'] = 3
param['eval_metric']='mlogloss'

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 5
bst = xgb.train(param, xg_train,num_round,evals=watchlist);
# get prediction
pred = bst.predict( xg_test );

xgb.cv(param, xg_train, num_round, nfold=3,
       metrics=['mlogloss'], seed = 0)

[0]	train-mlogloss:0.692844	test-mlogloss:0.713485
[1]	train-mlogloss:0.615574	test-mlogloss:0.649195
[2]	train-mlogloss:0.588324	test-mlogloss:0.627335
[3]	train-mlogloss:0.570136	test-mlogloss:0.613521
[4]	train-mlogloss:0.556119	test-mlogloss:0.605888


,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
0,0.708182,0.014134,0.688162,0.003330
1,0.643458,0.015117,0.610289,0.007806
2,0.620382,0.015579,0.582004,0.009094
3,0.613938,0.017532,0.569023,0.012251
4,0.608627,0.019875,0.562318,0.009976


In [51]:
xgb.plot_importance(booster=bst)
plt.show()

In [67]:
bst=xgb.train(param, full_xg_train, num_round,evals=watchlist)
test_response=bst.predict(full_xg_test)
xgb.cv(param, xg_train, num_round, nfold=3,
       metrics=['mlogloss'], seed = 0)

[0]	train-mlogloss:0.640880	test-mlogloss:0.646266
[1]	train-mlogloss:0.565513	test-mlogloss:0.573399
[2]	train-mlogloss:0.543859	test-mlogloss:0.543118
[3]	train-mlogloss:0.534413	test-mlogloss:0.531880
[4]	train-mlogloss:0.531563	test-mlogloss:0.529076
[5]	train-mlogloss:0.529124	test-mlogloss:0.526187
[6]	train-mlogloss:0.528490	test-mlogloss:0.525403
[7]	train-mlogloss:0.527273	test-mlogloss:0.523425
[8]	train-mlogloss:0.527267	test-mlogloss:0.523428
[9]	train-mlogloss:0.527264	test-mlogloss:0.523432


,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
0,0.688753,0.014217,0.656591,0.001498
1,0.636313,0.017328,0.580361,0.011052
2,0.616903,0.018551,0.552907,0.015418
3,0.609060,0.017938,0.542006,0.018086
4,0.606126,0.017244,0.531180,0.023546
5,0.605235,0.017456,0.527280,0.025777
6,0.603934,0.018257,0.524551,0.026027
7,0.603538,0.018039,0.520405,0.028249
8,0.603578,0.018058,0.517367,0.029859
9,0.603566,0.018041,0.517364,0.029861


In [68]:
#save predictions to required format 
#make it binary?
out=pd.DataFrame({'id':[],'predict_0':[],'predict_1':[],'predict_2':[]})
out['id']=join.loc['test']['id']
out['predict_0']=test_response.T[0]
out['predict_1']=test_response.T[1]
out['predict_2']=test_response.T[2]
out=out[['id','predict_0','predict_1','predict_2']]
out.to_csv('predictions.csv',index=False)

In [60]:
for column in join.columns:
    if column not in ['id','fault_severity']:
        join[column]=join[column].astype(np.float32)

In [120]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import RidgeClassifierCV, Perceptron, SGDClassifier, PassiveAggressiveClassifier
from sklearn.dummy import DummyClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import LinearSVC
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier, OutputCodeClassifier
from sklearn import grid_search

In [77]:
train_x, test_x, train_y, test_y = train_test_split(train_features, train_target, test_size=0.31, random_state=0)
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = t.time()
    #clf=OneVsRestClassifier(clf) #may want to remove for testing
    clf.fit(train_x, train_y)
    train_time = t.time() - t0
    print("Training time: %0.3fs" % train_time)

    t0 = t.time()
    y_pred_single = clf.predict(test_x)
    if (hasattr(clf,'predict_proba')):
        y_pred_dist = clf.predict_proba(test_x)
        logloss=log_loss(test_y,y_pred_dist)
        print("LogLoss: %0.3fs" % logloss)
    test_time = t.time() - t0
    print("Prediction time:  %0.3fs" % test_time)

    acc_score = accuracy_score(test_y, y_pred_single)
    print("Accuracy:   %0.3f" % acc_score)
    
    #if (hasattr(clf,'predict_proba')):
       #scores = cross_val_score(clf,train_features,train_target, scoring='log_loss')
        #loss_score=scores.mean()
        #print("CV LogLoss: %0.2f (+/- %0.2f)" % (loss_score, scores.std() * 2))
    
    print(classification_report(test_y, y_pred_single))
    clf_descr = str(clf).split('(')[0]
    
    if (hasattr(clf,'predict_proba')):
        return clf_descr, acc_score, logloss, train_time, test_time
    else:
        return clf_descr, acc_score, train_time, test_time

In [79]:
results = []
for clf, name in (
        (RidgeClassifierCV(), "Ridge Classifier"),
        (RandomForestClassifier(n_estimators=300), "Random forest"),
        (AdaBoostClassifier(), "AdaBoost"),
        (BaggingClassifier(), "Bagging"),
        (QuadraticDiscriminantAnalysis(), "DiscriminatnAnalysis")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))

Ridge Classifier
________________________________________________________________________________
Training: 
RidgeClassifierCV(alphas=(0.1, 1.0, 10.0), class_weight=None, cv=None,
         fit_intercept=True, normalize=False, scoring=None)
Training time: 4.643s
Prediction time:  0.038s
Accuracy:   0.727
             precision    recall  f1-score   support

        0.0       0.79      0.91      0.84      1476
        1.0       0.59      0.35      0.44       593
        2.0       0.49      0.55      0.52       220

avg / total       0.71      0.73      0.71      2289

Random forest
________________________________________________________________________________
Training: 
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verb

C:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:688: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [115]:
#test a classifier

# Fit the model to training data
#parameters={'n_estimators':[100,150]},'max_depth':[3]}
clf = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=200,verbose=1,presort=True,max_depth=7),n_jobs=-2)#,min_samples_leaf=10,min_samples_split=5)
#clf=grid_search.GridSearchCV(gbc, parameters,n_jobs=-2)
benchmark(clf)

________________________________________________________________________________
Training: 
OneVsRestClassifier(estimator=GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=7, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200, presort=True,
              random_state=None, subsample=1.0, verbose=1,
              warm_start=False),
          n_jobs=-2)
Training time: 92.315s
Prediction time:  0.573s
Accuracy:   0.739
             precision    recall  f1-score   support

        0.0       0.80      0.89      0.84      1476
        1.0       0.61      0.42      0.50       593
        2.0       0.55      0.60      0.58       220

avg / total       0.73      0.74      0.73      2289



('OneVsRestClassifier',
 0.73918741808650068,
 0.57045593736126277,
 92.31480956077576,
 0.5728719234466553)

In [122]:
#test a classifier

# Fit the model to training data
#parameters={'n_estimators':[100,150]},'max_depth':[3]}
clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=400,verbose=1,min_samples_split=10,\
                                                 oob_score=True,n_jobs=-2),n_jobs=-2)
#clf=grid_search.GridSearchCV(gbc, parameters,n_jobs=-2)
benchmark(clf)

________________________________________________________________________________
Training: 
OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-2,
            oob_score=True, random_state=None, verbose=1, warm_start=False),
          n_jobs=-2)
Training time: 31.099s

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks    


Prediction time:  1.980s
Accuracy:   0.751
             precision    recall  f1-score   support

        0.0       0.80      0.90      0.85      1476
        1.0       0.63      0.44      0.52       593
        2.0       0.59      0.56      0.57       220

avg / total       0.74      0.75      0.74      2289



('OneVsRestClassifier',
 0.75141983398864132,
 0.54245802948043198,
 31.099143743515015,
 1.9801154136657715)

In [110]:
scores = cross_val_score(clf,train_features,train_target, scoring='log_loss',n_jobs=-2,verbose=True)
print("LogLoss: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

LogLoss: -0.56 (+/- 0.01)


[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   44.6s finished


In [111]:
bestCLF=clf

In [112]:
bestCLF.fit(train_features,train_target)

[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-2)]: Done 194 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-2)]: Done 400 out of 400 | elapsed:   10.2s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=800,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-2,
            oob_score=True, random_state=None, verbose=1, warm_start=False)

In [113]:
test_response=bestCLF.predict_proba(test_features)

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.7s finished


In [5]:
#transform cat labels into digits
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for column in join.columns:
    if column not in notCat:
        join[column]=le.fit_transform(join[column])
    
join.head()

fault_severity     id  location  event_type  severity_type  \
sample                                                                 
train  0               1  14121       148          26              1   
       1               1  14121       148          26              1   
       2               1  14121       148          27              1   
       3               1  14121       148          27              1   
       4               0   9320      1027          26              1   

          resource_type  log_feature  volume  
sample                                        
train  0              2          237      19  
       1              2          148      19  
       2              2          237      19  
       3              2          148      19  
       4              2          240     200